In [1]:
import numpy as np
import os
import pandas as pd
import requests
import hmac
import hashlib
import time
from urllib.parse import quote
import json
import pyodbc
import statsmodels.api as sm
from datetime import datetime, UTC
from pandas.tseries.offsets import QuarterBegin
from pandas.tseries.offsets import MonthEnd
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
from scipy.stats import norm
from scipy.optimize import root_scalar
from scipy.optimize import brentq
from scipy.stats import multivariate_normal
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt

In [2]:
# set working directory
os.chdir('../')
# print working directory
os.getcwd()

"c:\\Users\\XiaoA1\\OneDrive - Moody's\\COE - Credit Analytics-SQ - Unified ST Framework - Documents\\Sovereign Macro\\2 - Research Code"

In [3]:
version = '052025'

# Download Data from DataBuffet

In [4]:
#####
# Setup:
# 1. Store your access key, encryption key.
# Get your keys at:
# https://www.economy.com/myeconomy/api-key-info
ACC_KEY = "FF91D2DD-FB35-4664-BDA5-E165A718941D"
ENC_KEY = "3F2579A0-99EF-487C-B0D9-0ECA49DFC46E"


def get_series(mnemonic, acc_key, enc_key, freq="0", trans="0", vintage='latest'):
    if vintage == 'latest':
        vintage = datetime.now(UTC).strftime("%Y%m")
    
    api_command = f"series?m={quote(mnemonic)}&freq={freq}&trans={trans}&vintage={vintage}"
    url = f"https://api.economy.com/data/v1/{api_command}"
    print(url)
    
    time_stamp = datetime.now(UTC).strftime("%Y-%m-%dT%H:%M:%SZ")
    hash_msg = f"{acc_key}{time_stamp}"
    signature = hmac.new(enc_key.encode(), hash_msg.encode(), hashlib.sha256).hexdigest()
    
    time.sleep(1)
    headers = {
        "AccessKeyId": acc_key,
        "Signature": signature,
        "TimeStamp": time_stamp
    }
    
    response = requests.get(url, headers=headers)
    
    series = response.json()
    print(series.get('description'))
    return series

In [67]:
sov_list = pd.read_excel('Output/1.sov_monthly_summary_'+version+'.xlsx')
sov_list = sov_list[sov_list['Included']==True]
sov_list.loc[sov_list['cinc'] == 'SCG', 'cinc'] = 'SRB'
sov_list

,pid,entityName,cinc,region,Nmonth,Nquarter,cdsiedf5 unique value count,start,end,cdsiedf5 unique value count (%),Included
1,S10159,"AUSTRALIA, GOVERNMENT OF",AUS,Australia,233,78,233,2006-01-31,2025-05-31,1.000000,True
2,S10539,"AUSTRIA, GOVERNMENT OF",AUT,Europe,233,78,233,2006-01-31,2025-05-31,1.000000,True
3,S10368,"BELGIUM, GOVERNMENT OF",BEL,Europe,233,78,233,2006-01-31,2025-05-31,1.000000,True
4,S10540,"BULGARIA, GOVERNMENT OF",BGR,Eastern Europe,233,78,233,2006-01-31,2025-05-31,1.000000,True
5,S10367,"BAHRAIN, GOVERNMENT OF",BHR,Middle East,233,78,233,2006-01-31,2025-05-31,1.000000,True
...,...,...,...,...,...,...,...,...,...,...,...
63,S10577,"TURKEY, GOVERNMENT OF",TUR,Middle East,233,78,233,2006-01-31,2025-05-31,1.000000,True
65,S10711,"UNITED STATES OF AMERICA, GOVERNMENT OF",USA,North America,233,78,229,2006-01-31,2025-05-31,0.982833,True
66,S10578,"VENEZUELA, GOVERNMENT OF",VEN,Latin America,143,48,143,2006-01-31,2017-11-30,0.613734,True
67,S10622,"VIETNAM, GOVERNMENT OF",VNM,Asia,233,78,233,2006-01-31,2025-05-31,1.000000,True


In [68]:
mv_types = [
	"GDP",
	"Unemployment Rate",
	"Equity",
	"FX",
	"Inflation",
    "Government 10Y Bond Rate",
    "Government Consumption",
    "Net Exports",
    # "External Debt",
    # "External Debt World Bank",
    # "External Debt EBA 2025 Baseline",
	"Debt to GDP Ratio"
]

mnemonics_general = [
    "FGDPL$Q.I",
    "FLBRQ.I",
    "FSTOCKPQ.I",
    "FTFXIUSAQ.I",
    "FCPIQ.I",
    "FRGT10YQ.I",
    "FGD$Q.I",
    "FNETEXGSD$Q.I",
    # "ECBDBETOTUQ.I",
    # "WBKDODDECTCDARUQ.I",
    # "FEXTDEBTDQ_EBABL.I",
    # "ECBDB%DGRRUQ.I"
    "FGGDEBTGDPQ.I"
]
sov_list[mv_types] = np.nan

In [69]:
data_all = pd.DataFrame()
for cinc in sov_list['cinc']:
    for mv_type, mnemonic in zip(mv_types, mnemonics_general):
        mnemonic_cinc = mnemonic + cinc
        my_data = get_series(mnemonic_cinc, ACC_KEY, ENC_KEY)
        if my_data['data'] == None:
            continue
        start_date = datetime.strptime(my_data['startDate'][:10], "%Y-%m-%d")
        end_date = datetime.strptime(my_data['endDate'][:10], "%Y-%m-%d")
        if my_data['data']['freq'] == 'MONTH':
            dates = pd.date_range(start=start_date, end=end_date, freq='M') + MonthEnd(0)
        else:
            dates = pd.date_range(start=start_date, end=end_date, freq='QE') + QuarterBegin(startingMonth=1) - pd.DateOffset(days=1)
        data = pd.DataFrame({
            'yyyyqq': dates,
            'ma.ticker': mnemonic_cinc,
            'value': pd.to_numeric(my_data['data']['data'])
        })
        sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_cinc
        data_all = pd.concat([data_all, data], ignore_index = True)

data_all.to_csv(f'Output/2_mv_raw_data.csv', index = False)

https://api.economy.com/data/v1/series?m=FGDPL%24Q.IAUS&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): National Accounts: Real Gross Domestic Product [GDP], (Bil. Ch. FY2023 AUD, SAAR)
https://api.economy.com/data/v1/series?m=FLBRQ.IAUS&freq=0&trans=0&vintage=202505


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_22740\2675099222.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'FGDPL$Q.IAUS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_cinc


Baseline Scenario (May 2025): Labor Force Survey: Unemployment Rate, (%, SA)
https://api.economy.com/data/v1/series?m=FSTOCKPQ.IAUS&freq=0&trans=0&vintage=202505


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_22740\2675099222.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'FLBRQ.IAUS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_cinc


Baseline Scenario (May 2025): Stock Market: All Ordinaries Index, (Index Jan1980=500, NSA)
https://api.economy.com/data/v1/series?m=FTFXIUSAQ.IAUS&freq=0&trans=0&vintage=202505


...

Baseline Scenario (May 2025): National Accounts: Real Net Exports of Goods and Services, (Bil. 2017 USD, SAAR)
https://api.economy.com/data/v1/series?m=FGGDEBTGDPQ.IAUS&freq=0&trans=0&vintage=202505


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_22740\2675099222.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'FNETEXGSD$Q.IAUS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_cinc


Baseline Scenario (May 2025): General government debt to GDP ratio, (%, SA)
https://api.economy.com/data/v1/series?m=FGDPL%24Q.IAUT&freq=0&trans=0&vintage=202505


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_22740\2675099222.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'FGGDEBTGDPQ.IAUS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_cinc


Baseline Scenario (May 2025): National Accounts: Real Gross Domestic Product [GDP], (Bil. Ch. 2015 EUR, WDASAAR)
https://api.economy.com/data/v1/series?m=FLBRQ.IAUT&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): Labor Force Survey: Unemployment Rate, (%, SA)
https://api.economy.com/data/v1/series?m=FSTOCKPQ.IAUT&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): Stock Market: ATX Index, (Index, NSA)
https://api.economy.com/data/v1/series?m=FTFXIUSAQ.IAUT&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): Nominal Bilateral Exchange Rate, (USD per EUR, NSA)
https://api.economy.com/data/v1/series?m=FCPIQ.IAUT&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): Consumer Price Index, (Index 2020=100, SA)
https://api.economy.com/data/v1/series?m=FRGT10YQ.IAUT&freq=0&trans=0&vintage=202505
Baseline Scenario (May 2025): Interest Rate: Government Bond Yield - 10 Year, (% p.a., NSA)
https://api.economy.com/data/v1/series?m=FGD%24Q.IAUT&freq=0&trans=0&vintag

In [70]:
display(sov_list[['cinc', 'entityName'] + mv_types].style.set_properties(**{'max-height': 'none', 'max-width': 'none'}))

,cinc,entityName,GDP,Unemployment Rate,Equity,FX,Inflation,Government 10Y Bond Rate,Government Consumption,Net Exports,Debt to GDP Ratio
1,AUS,"AUSTRALIA, GOVERNMENT OF",FGDPL$Q.IAUS,FLBRQ.IAUS,FSTOCKPQ.IAUS,FTFXIUSAQ.IAUS,FCPIQ.IAUS,FRGT10YQ.IAUS,FGD$Q.IAUS,FNETEXGSD$Q.IAUS,FGGDEBTGDPQ.IAUS
2,AUT,"AUSTRIA, GOVERNMENT OF",FGDPL$Q.IAUT,FLBRQ.IAUT,FSTOCKPQ.IAUT,FTFXIUSAQ.IAUT,FCPIQ.IAUT,FRGT10YQ.IAUT,FGD$Q.IAUT,FNETEXGSD$Q.IAUT,FGGDEBTGDPQ.IAUT
3,BEL,"BELGIUM, GOVERNMENT OF",FGDPL$Q.IBEL,FLBRQ.IBEL,FSTOCKPQ.IBEL,FTFXIUSAQ.IBEL,FCPIQ.IBEL,FRGT10YQ.IBEL,FGD$Q.IBEL,FNETEXGSD$Q.IBEL,FGGDEBTGDPQ.IBEL
4,BGR,"BULGARIA, GOVERNMENT OF",FGDPL$Q.IBGR,FLBRQ.IBGR,FSTOCKPQ.IBGR,FTFXIUSAQ.IBGR,FCPIQ.IBGR,FRGT10YQ.IBGR,FGD$Q.IBGR,FNETEXGSD$Q.IBGR,FGGDEBTGDPQ.IBGR
5,BHR,"BAHRAIN, GOVERNMENT OF",FGDPL$Q.IBHR,FLBRQ.IBHR,FSTOCKPQ.IBHR,FTFXIUSAQ.IBHR,FCPIQ.IBHR,nan,FGD$Q.IBHR,FNETEXGSD$Q.IBHR,FGGDEBTGDPQ.IBHR
6,BRA,"BRAZIL, GOVERNMENT OF",FGDPL$Q.IBRA,FLBRQ.IBRA,FSTOCKPQ.IBRA,FTFXIUSAQ.IBRA,FCPIQ.IBRA,FRGT10YQ.IBRA,FGD$Q.IBRA,FNETEXGSD$Q.IBRA,FGGDEBTGDPQ.IBRA
7,CAN,"CANADA, GOVERNMENT OF",FGDPL$Q.ICAN,FLBRQ.ICAN,FSTOCKPQ.ICAN,FTFXIUSAQ.ICAN,FCPIQ.ICAN,FRGT10YQ.ICAN,FGD$Q.ICAN,FNETEXGSD$Q.ICAN,FGGDEBTGDPQ.ICAN
8,CHE,"SWITZERLAND, GOVERNMENT OF",FGDPL$Q.ICHE,FLBRQ.ICHE,FSTOCKPQ.ICHE,FTFXIUSAQ.ICHE,FCPIQ.ICHE,FRGT10YQ.ICHE,FGD$Q.ICHE,FNETEXGSD$Q.ICHE,FGGDEBTGDPQ.ICHE
9,CHL,"CHILE, GOVERNMENT OF",FGDPL$Q.ICHL,FLBRQ.ICHL,FSTOCKPQ.ICHL,FTFXIUSAQ.ICHL,FCPIQ.ICHL,FRGT10YQ.ICHL,FGD$Q.ICHL,FNETEXGSD$Q.ICHL,FGGDEBTGDPQ.ICHL
10,CHN,"CHINA, GOVERNMENT OF",FGDPL$Q.ICHN,FLBRQ.ICHN,FSTOCKPQ.ICHN,FTFXIUSAQ.ICHN,FCPIQ.ICHN,FRGT10YQ.ICHN,FGD$Q.ICHN,FNETEXGSD$Q.ICHN,FGGDEBTGDPQ.ICHN


In [5]:
sov_list[['cinc', 'entityName'] + mv_types].to_csv("Output/2.sov_mv_mnemonics.csv", index=False)

NameError: name 'sov_list' is not defined

In [4]:
data_all = pd.read_csv(f'Output/2_mv_raw_data.csv')
data_all['yyyyqq'] = pd.to_datetime(data_all['yyyyqq'])
mnemonics_list = pd.read_csv("Output/2.sov_mv_mnemonics.csv")

In [8]:
mnemonics_long = mnemonics_list.melt(
    id_vars=['cinc', 'entityName'],
    var_name='mv_type',
    value_name='mnemonics'
)
mnemonics_long = mnemonics_long.dropna(subset=['mnemonics'])
mnemonics_long.to_csv("Output/2.sov_mv_mnemonics_long.csv", index=False)
mnemonics_long

,cinc,entityName,mv_type,mnemonics
0,AUS,"AUSTRALIA, GOVERNMENT OF",GDP,FGDPL$Q.IAUS
1,AUT,"AUSTRIA, GOVERNMENT OF",GDP,FGDPL$Q.IAUT
2,BEL,"BELGIUM, GOVERNMENT OF",GDP,FGDPL$Q.IBEL
3,BGR,"BULGARIA, GOVERNMENT OF",GDP,FGDPL$Q.IBGR
4,BHR,"BAHRAIN, GOVERNMENT OF",GDP,FGDPL$Q.IBHR
...,...,...,...,...
543,TUN,"TUNISIA, GOVERNMENT OF",Debt to GDP Ratio,FGGDEBTGDPQ.ITUN
544,TUR,"TURKEY, GOVERNMENT OF",Debt to GDP Ratio,FGGDEBTGDPQ.ITUR
545,USA,"UNITED STATES OF AMERICA, GOVERNMENT OF",Debt to GDP Ratio,FGGDEBTGDPQ.IUSA
547,VNM,"VIETNAM, GOVERNMENT OF",Debt to GDP Ratio,FGGDEBTGDPQ.IVNM


In [44]:
data = data_all[data_all['ma.ticker'].isin(mnemonics['mnemonics'].values)]
# data = pd.merge(data, mnemonics, left_on='ma.ticker', right_on='mnemonics', how='left')
data

,yyyyqq,ma.ticker,value
2230,1988-03-31,FGDPL$Q.IAUT,193.882404
2231,1988-06-30,FGDPL$Q.IAUT,197.451678
2232,1988-09-30,FGDPL$Q.IAUT,200.741791
2233,1988-12-31,FGDPL$Q.IAUT,203.114660
2234,1989-03-31,FGDPL$Q.IAUT,204.784641
...,...,...,...
4174,2023-12-31,ECBDB%DGRRUQ.IAUT,152.538000
4175,2024-03-31,ECBDB%DGRRUQ.IAUT,157.511000
4176,2024-06-30,ECBDB%DGRRUQ.IAUT,157.604000
4177,2024-09-30,ECBDB%DGRRUQ.IAUT,161.124000


In [60]:
for cinc in mnemonics_list['cinc']:
    mnemonics = mnemonics_list[mnemonics_list['cinc'] == cinc].T.reset_index()
    mnemonics.columns = ['name','mnemonics']
    data = data_all[data_all['ma.ticker'].isin(mnemonics['mnemonics'].values)]
    data = pd.merge(data, mnemonics, left_on='ma.ticker', right_on='mnemonics', how='left')
    data = data.pivot(index='yyyyqq', columns='name', values='value')
    if 'Debt to GDP Ratio' in data.columns:
        data['GDP x Debt to GDP Ratio'] = data['GDP'] * data['Debt to GDP Ratio']
    # Only select columns that exist in the data
    cols_to_plot = [col for col in ['GDP x Debt to GDP Ratio', 'External Debt', 'External Debt World Bank', 'External Debt EBA 2025 Baseline'] if col in data.columns]
    data_plot = data[cols_to_plot].copy().dropna()
    data_plot = data_plot[data_plot.index < '2024-01-01']
    for col in cols_to_plot:
        data_plot[col] = data_plot[col] / data_plot[col].mean()
    data_plot = data_plot.reset_index()
    plt.figure(figsize=(10, 6))
    for col in cols_to_plot:
        plt.plot(data_plot['yyyyqq'], data_plot[col], label=col)
    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.grid()
    plt.savefig(f'Plot/2.external_debt_comparison/{cinc}.png')
    plt.close()
    


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_22740\3152723971.py:19: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
